In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from qiskit_braket_provider import BraketProvider
from qiskit.providers import Backend

provider = BraketProvider()
aria_2 : Backend = provider.get_backend("Aria 2")

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

hamiltonian: FermionicOp = (
    PySCFDriver(
        atom="H 0 0 0; H 0 0 0.735",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    .run()
    .hamiltonian.second_q_op()
)

from qiskit.quantum_info import Statevector
from qiskit_nature.second_q.mappers.fermionic_mapper import FermionicMapper
from statistics import mean, variance

def process(data: dict[str, int], mapper: FermionicMapper, hamiltonian):
    hamiltonian = mapper.map(hamiltonian)

    energies = []

    for state, n in data.items():
        initial_state = Statevector.from_label(state[::-1])
        exp_energy = initial_state.expectation_value(hamiltonian)  # type: ignore
        print(exp_energy, n)

        for _ in range(n):
            energies.append(exp_energy.real)

    return (
        mean(energies),
        variance(energies),
    )

In [5]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper

jw_data = {
    "0000": 0.07066666666666667,
    "1000": 0.098,
    "0100": 0.04666666666666667,
    "1100": 0.035,
    "0010": 0.255,
    "1010": 0.09466666666666666,
    "0110": 0.08033333333333334,
    "1110": 0.054,
    "0001": 0.035,
    "1001": 0.06333333333333334,
    "0101": 0.017,
    "1101": 0.024333333333333332,
    "0011": 0.035666666666666666,
    "1011": 0.035666666666666666,
    "0111": 0.027,
    "1111": 0.027666666666666666,
}

jw_data = {k: round(v * 3000) for k, v in jw_data.items()}

process(jw_data, BravyiKitaevMapper(), hamiltonian)

(3.3306690738754696e-16+0j) 212
(-1.2445845498133266+0j) 294
(-0.24521829183026567+0j) 140
(-1.0636533500290954+0j) 105
(-1.2445845498133266+0j) 765
(0.2142782384194732+0j) 284
(-1.0636533500290954+0j) 241
(-1.8369679912029837+0j) 162
(-0.4718960072811415+0j) 105
(-0.3533251041071544+0j) 190
(-0.4718960072811416+0j) 51
(-1.2563390730032498+0j) 73
(-1.2563390730032498+0j) 107
(-1.1606317377577642+0j) 107
(-0.35332510410715445+0j) 81
(-1.1606317377577642+0j) 83


(-0.8577104655819423, 0.3339685176741624)

# Jordan-Wigner

In [2]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from utility import Execution

jwsim = Execution(hamiltonian, JordanWignerMapper())
jwjob = aria_2.run(jwsim.circuit, shots=3000)

Hamiltonian simulation:
  Solve ground state:
    Ground energy = -1.85727503020238
  Construct and transpile Pauli evolution:
    Time duration    = 1.0
    Basis gates      = cx, u3
    Synthesis method = LieTrotter
  Circuit summary:
    Depth = 89
    Gates = 57 U3, 50 CX, 4 MEASURE


In [1]:
process(jwjob.result().data()["counts"], JordanWignerMapper(), hamiltonian)

NameError: name 'process' is not defined

# Bravyi-Kitaev

In [3]:
from qiskit_nature.second_q.mappers import BravyiKitaevMapper
from utility import Execution

bksim = Execution(hamiltonian, BravyiKitaevMapper())
bkjob = aria_2.run(bksim.circuit, shots=3000)